## Import Libraries

In [1]:
#!/usr/bin/env python
# coding: utf-8
import os
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json

In [2]:
# Variables
season = "2020-21"

## Import files

In [3]:
all_events = pd.read_csv(r'./data/' + season + '/Cleaned_data/all_events.csv', parse_dates= ["deadline_time"])
all_players = pd.read_csv(r'./data/' + season + '/Merged_data/all_players_merged.csv', parse_dates= ["news_added"]) 



## Functions

In [4]:
#### Connect to FPL API
def get(url):
    response = requests.get(url)
    return json.loads(response.content)

## My Team

In [5]:
#### Get my fantasy team lineup
now = datetime.now()
my_team_id = '1341730'
current_gw = all_events[all_events['deadline_time'] < now]['gameweek_id'].max()

my_team_url = 'https://fantasy.premierleague.com/api/entry/{}/event/{}/picks/'.format(my_team_id,str(current_gw))

my_team = get(my_team_url)['picks']
my_team = pd.DataFrame(my_team)


In [6]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round',
       'player_code', 'cost_change_event', 'cost_change_event_fall',
       'cost_change_start', 'cost_change_start_fall', 'dreamteam_count',
       'player_type_id', 'ep_next', 'ep_this', 'event_points', 'form',
       'player_id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo',
       'PPG', 'selected_by_percent', 'status', 'team_id', 'team_code',
       'total_points', 'transfers_in_event', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'total_minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'influence_rank', 'influence_rank_type',
       'creativity_rank', 'creativity_rank_type', 'threat_rank',
       'threat_rank_type', 'ict_index_rank', 'ict_index_rank_type',
       'corne

In [7]:
#all_players = all_players[['chance_of_playing_next_round', 'chance_of_playing_this_round', 'position_id', 'ep_next','ep_this', 'player_id', 'player_news', 'news_added', 'player_price', 'selected_by_percent', 'player_status','team_id', 'web_name']]

all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,...,penalties_order,penalties_text,PP90,PPMM,VAPM,team_strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,0.0,0.0,37605,0,0,-2,2,0,3,0.0,...,NaN,NaN,NaN,0.000000,-0.294118,4,ARS,1240,1250,MID
1,100.0,100.0,54694,-1,1,-3,3,0,3,2.0,...,1.0,NaN,3.333333,0.282051,0.111111,4,ARS,1240,1250,MID
2,NaN,NaN,84450,0,0,-1,1,0,3,1.2,...,NaN,NaN,2.211302,0.314815,-0.055556,4,ARS,1240,1250,MID
3,NaN,NaN,195735,0,0,-3,3,0,3,2.5,...,2.0,NaN,5.346535,0.259740,0.000000,4,ARS,1240,1250,MID
4,0.0,0.0,198849,-1,1,-3,3,0,3,0.0,...,NaN,NaN,NaN,0.000000,-0.425532,4,ARS,1240,1250,MID


In [8]:
my_team

,element,position,multiplier,is_captain,is_vice_captain
0,363,1,1,False,False
1,255,2,1,False,True
2,259,3,1,False,False
3,232,4,1,False,False
4,461,5,1,False,False
5,390,6,1,False,False
6,508,7,1,False,False
7,276,8,2,True,False
8,377,9,1,False,False
9,117,10,1,False,False


In [9]:
my_team = my_team.merge(all_players, how = 'left', left_on="element", right_on="player_id")
my_team.drop(["element","position"], axis = 1, inplace = True)

In [10]:
my_team.sort_values(by=['player_type_id' , 'selected_by_percent'])['web_name']



11              Nyland
0             McCarthy
12              Kilman
3             Chilwell
1            Robertson
4                Saïss
2     Alexander-Arnold
14             Ritchie
13            Bissouma
7             Sterling
6            Rodríguez
5                  Son
8                Adams
10             Jiménez
9               Werner
Name: web_name, dtype: object

In [11]:
my_team.to_csv (r'./data/' + season + '/Team_data/my_team.csv', index = False, header=True)